In [2]:
# Standard imports
import numpy  as np
import pandas as pd

import itertools
import random
import math  
import copy

from pprint import pprint  # nicer printing

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Other NLP
import re
import spacy
import nltk
from nltk.corpus import stopwords

# General Plotting
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
%matplotlib inline  
import seaborn as sns
sns.set(style="white")

# ignore some warnings 
import warnings
warnings.filterwarnings('ignore')

# Set the maximum number of rows displayed by pandas
pd.options.display.max_rows = 1000

In [41]:
def baseline_model(y_train, y_model):   
    y_train_df = pd.DataFrame(y_train)
    most_freq = np.argmax(y_train_df.value_counts())

    y_hat = np.array([int(np.round(y_train_df.value_counts().index[most_freq]))]*len(y_train))
    y_hat_df = pd.DataFrame((y_hat), columns = ["prediction"])
    y_hat_df["truth"] = y_train
    #y_hat_df.head()

    y_hat = [int(np.round(y_train_df.value_counts().index[most_freq]))]*len(y_test)
    cm = confusion_matrix(y_test, y_hat)
    acc =np.diagonal(cm).sum()/cm.sum()

    auc = roc_auc_score(y_test, y_hat)
    
    return y_hat_df, acc, auc

In [3]:
audible = pd.read_csv('Audible_Dataset_final.csv')

In [4]:
audible.shape

(2269, 110)

In [5]:
audible.head()

,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price,...,Review 91,Review 92,Review 93,Review 94,Review 95,Review 96,Review 97,Review 98,Review 99,Review100
0,Bamboozled by Jesus,How God Tricked Me into the Life of My Dreams,Yvonne Orji,Yvonne Orji,6 hrs and 31 mins,Unabridged Audiobook,Biographies & Memoirs,5,47.0,$29.65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sixth Realm Part 1,"A LitRPG Fantasy Series (The Ten Realms, Book 6)",Michael Chatfield,Neil Hellegers,13 hrs and 33 mins,Unabridged Audiobook,Science Fiction & Fantasy,4.5,98.0,$24.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Go Tell the Bees That I Am Gone,"Outlander, Book 9",Diana Gabaldon,Davina Porter,27 hrs and 30 mins,Unabridged Audiobook,Science Fiction & Fantasy,NaN,NaN,$41.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,How the Word Is Passed,A Reckoning with the History of Slavery Across...,Clint Smith,Clint Smith,10 hrs and 7 mins,Unabridged Audiobook,History,NaN,NaN,$29.65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,The Devil May Dance,A Novel,Jake Tapper,Rob Shapiro,10 hrs and 12 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.5,108.0,$29.65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
list(audible.columns.values)

['Book Title',
 'Book Subtitle',
 'Book Author',
 'Book Narrator',
 'Audio Runtime',
 'Audiobook_Type',
 'Categories',
 'Rating',
 'Total No. of Ratings',
 'Price',
 'Review 1',
 'Review 2',
 'Review 3',
 'Review 4',
 'Review 5',
 'Review 6',
 'Review 7',
 'Review 8',
 'Review 9',
 'Review 10',
 'Review 11',
 'Review 12',
 'Review 13',
 'Review 14',
 'Review 15',
 'Review 16',
 'Review 17',
 'Review 18',
 'Review 19',
 'Review 20',
 'Review 21',
 'Review 22',
 'Review 23',
 'Review 24',
 'Review 25',
 'Review 26',
 'Review 27',
 'Review 28',
 'Review 29',
 'Review 30',
 'Review 31',
 'Review 32',
 'Review 33',
 'Review 34',
 'Review 35',
 'Review 36',
 'Review 37',
 'Review 38',
 'Review 39',
 'Review 40',
 'Review 41',
 'Review 42',
 'Review 43',
 'Review 44',
 'Review 45',
 'Review 46',
 'Review 47',
 'Review 48',
 'Review 49',
 'Review 50',
 'Review 51',
 'Review 52',
 'Review 53',
 'Review 54',
 'Review 55',
 'Review 56',
 'Review 57',
 'Review 58',
 'Review 59',
 'Review 60',
 'Re

In [7]:
nan_values = audible.isna()
nan_columns = nan_values.any()

columns_with_nan = audible.columns[nan_columns].tolist()
print(columns_with_nan)

['Book Subtitle', 'Book Narrator', 'Audiobook_Type', 'Categories', 'Rating', 'Total No. of Ratings', 'Review 1', 'Review 2', 'Review 3', 'Review 4', 'Review 5', 'Review 6', 'Review 7', 'Review 8', 'Review 9', 'Review 10', 'Review 11', 'Review 12', 'Review 13', 'Review 14', 'Review 15', 'Review 16', 'Review 17', 'Review 18', 'Review 19', 'Review 20', 'Review 21', 'Review 22', 'Review 23', 'Review 24', 'Review 25', 'Review 26', 'Review 27', 'Review 28', 'Review 29', 'Review 30', 'Review 31', 'Review 32', 'Review 33', 'Review 34', 'Review 35', 'Review 36', 'Review 37', 'Review 38', 'Review 39', 'Review 40', 'Review 41', 'Review 42', 'Review 43', 'Review 44', 'Review 45', 'Review 46', 'Review 47', 'Review 48', 'Review 49', 'Review 50', 'Review 51', 'Review 52', 'Review 53', 'Review 54', 'Review 55', 'Review 56', 'Review 57', 'Review 58', 'Review 59', 'Review 60', 'Review 61', 'Review 62', 'Review 63', 'Review 64', 'Review 65', 'Review 66', 'Review 67', 'Review 68', 'Review 69', 'Review 70'

In [8]:
i =0
while i < 9:
    for name in audible.columns:
        audible = audible[audible[name].notna()]
        i=i+1

In [9]:
audible.shape

(627, 110)

In [22]:
data = audible.iloc[:, 0:10].copy() # To avoid the case where changing df1 also changes df

In [23]:
data.head()

,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price
12,The Bomber Mafia,"A Dream, a Temptation, and the Longest Night o...",Malcolm Gladwell,Malcolm Gladwell,5 hrs and 14 mins,Unabridged Audiobook,History,4.7,1923.0,$22.40
22,What Happened to You?,"Conversations on Trauma, Resilience, and Healing",Oprah Winfrey,Bruce D. Perry,8 hrs and 27 mins,Unabridged Audiobook,Health & Wellness,4.9,2048.0,$20.99
23,The Last Thing He Told Me,A Novel,Laura Dave,Rebecca Lowman,8 hrs and 49 mins,Unabridged Audiobook,Literature & Fiction,4.5,4460.0,$22.67
30,Sapiens,A Brief History of Humankind,Yuval Noah Harari,Derek Perkins,15 hrs and 17 mins,Unabridged Audiobook,Politics & Social Sciences,4.7,43410.0,$34.22
31,Never Split the Difference,Negotiating as if Your Life Depended on It,Chris Voss,Michael Kramer,8 hrs and 7 mins,Unabridged Audiobook,Business & Careers,4.8,50427.0,$27.37


In [29]:
columns = data.columns
columns

Index(['Book Title', 'Book Subtitle', 'Audio Runtime', 'Rating',
       'Total No. of Ratings', 'Price', 'Book Author_A. J. Finn',
       'Book Author_Abi Daré', 'Book Author_Adam Grant',
       'Book Author_Adam M. Grant PhD',
       ...
       'Categories_Literature & Fiction', 'Categories_Money & Finance',
       'Categories_Mystery, Thriller & Suspense',
       'Categories_Politics & Social Sciences',
       'Categories_Relationships, Parenting & Personal Development',
       'Categories_Religion & Spirituality', 'Categories_Romance',
       'Categories_Science & Engineering',
       'Categories_Science Fiction & Fantasy', 'Categories_Teen'],
      dtype='object', length=956)

In [30]:
#Dummy variables for author, narrator, audiobook type, categories
data = pd.get_dummies(data = data, columns = ['Book Author', 'Book Narrator', 'Audiobook_Type', 'Categories'])

KeyError: "None of [Index(['Book Author', 'Book Narrator', 'Audiobook_Type', 'Categories'], dtype='object')] are in the [columns]"

In [43]:
# Select features 
features = list(set(list(data.columns)) - set(["Price"]))
X = data.loc[:, features]